In [1]:
import psycopg2
import pandas as pd
import numpy as np

In [2]:
conn = psycopg2.connect(
    host="localhost",
    dbname="postgres",
    user="postgres",
    password="postgres"
)

In [3]:
cur = conn.cursor()

In [4]:
cur.execute("""
        select bank_id, wallet_nickname, wight, swtc+swtv total
        from(
            select bank_id, wallet_nickname, sum(wight) wight , sum(transaction_value) swtv, sum(transaction_cost) swtc
            from(
                select bank_id, wallet_nickname, sum(new_count) wight, transaction_cost, transaction_value
                from(
                    select *
                    from(
                        select bank_id, wallet_nickname,  count(count) new_count, transaction_cost, transaction_value
                        from data_dataset
                        where count = 0
                        group by bank_id, wallet_nickname, transaction_cost, transaction_value) as b0
 
                    union all
                    select *
                    from(
                        select bank_id, wallet_nickname,  count new_count, transaction_cost, transaction_value
                        from data_dataset
                        where count != 0) as b1) as b2
                group by bank_id, wallet_nickname, transaction_cost, transaction_value) as b3
            group by bank_id, wallet_nickname) as b4
            where bank_id = 13335
            order by total desc
            """)

In [5]:
result = cur.fetchall()
pd.DataFrame(result)

,0,1,2,3
0,13335,None,620329,57183974007
1,13335,ed0c3917e9380b3,18147,502134277
2,13335,862dcf18a22fc92,191,421625509
3,13335,57b9945eac5a433,870,257120247
4,13335,9a42b3c27007806,98,250447579
...,...,...,...,...
11971,13335,eda57da6176c589,1,300
11972,13335,b8344b280831256,1,300
11973,13335,0e5c226be7b95b4,2,300
11974,13335,26247a16670b7a3,1,300


In [6]:
data = pd.read_csv('tranasaction_dataset.csv', parse_dates=['time_internal'])
data = data.replace(np.nan,0)

In [7]:
tmp_data = data[data['count'] == 0]
dg1 = tmp_data.groupby(by=['bank_id', 'wallet_nickname', 'transaction_cost', 'transaction_value']).agg({'count': 'count'}).reset_index()

In [8]:
tmp_data2 = data[data['count'] != 0]
dg2 = tmp_data2[['count', 'bank_id', 'wallet_nickname', 'transaction_cost', 'transaction_value']]

In [9]:
dg = pd.concat([dg1, dg2])

In [10]:
dg = dg.groupby(by=['bank_id', 'wallet_nickname', 'transaction_cost', 'transaction_value']).agg({'count': 'sum'}).reset_index()

In [11]:
dg = dg.groupby(by=['bank_id', 'wallet_nickname']).agg({
    'count' : 'sum',
    'transaction_cost' : 'sum',
    'transaction_value' : 'sum'
}).reset_index()

In [12]:
dg['total_value'] = dg['transaction_cost'] + dg['transaction_value']
dg = dg.drop(['transaction_cost', 'transaction_value'], axis='columns')

In [13]:
your_bank_id = 13335
dg[dg['bank_id'] == your_bank_id].sort_values('total_value', ascending=False)

,bank_id,wallet_nickname,count,total_value
37208,13335,0,620329,57183974007
48274,13335,ed0c3917e9380b3,18147,502134277
43413,13335,862dcf18a22fc92,191,421625509
41281,13335,57b9945eac5a433,870,257120247
44417,13335,9a42b3c27007806,98,250447579
...,...,...,...,...
48044,13335,e7d740e79859f58,5,300
48303,13335,eda57da6176c589,1,300
45852,13335,b8344b280831256,1,300
37860,13335,0e5c226be7b95b4,2,300
